In [1]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

In [16]:
data_dir = 'private/tomatect'

In [17]:
data_generator = ImageDataGenerator(
    rescale=1./255,  # Rescaling intensitas piksel menjadi rentang [0,1]
    rotation_range=40,  # Rotasi citra hingga 40 derajat
    width_shift_range=0.2,  # Perpindahan horizontal sebesar 20% lebar citra
    height_shift_range=0.2,  # Perpindahan vertikal sebesar 20% tinggi citra
    shear_range=0.2,  # Pemotongan citra sebesar 20%
    zoom_range=0.2,  # Perbesaran citra sebesar 20%
    horizontal_flip=True,  # Membalik citra secara horizontal
    fill_mode='nearest'  # Mengisi piksel yang hilang dengan piksel terdekat
)

In [18]:
batch_size = 10
input_shape = (150, 150, 3)

In [19]:
data_generator = data_generator.flow_from_directory(
    data_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

Found 250 images belonging to 5 classes.


In [20]:
# Membagi data menjadi set pelatihan, validasi, dan pengujian
total_samples = len(data_generator.filenames)
train_samples = int(total_samples * 0.8)
val_samples = int(total_samples * 0.15)
test_samples = total_samples - train_samples - val_samples

train_data = data_generator.next()
train_images, train_labels = train_data[0], train_data[1]
train_labels = np.argmax(train_labels, axis=1)

val_data = data_generator.next()
val_images, val_labels = val_data[0], val_data[1]
val_labels = np.argmax(val_labels, axis=1)

test_data = data_generator.next()
test_images, test_labels = test_data[0], test_data[1]
test_labels = np.argmax(test_labels, axis=1)


In [25]:
from keras.utils import to_categorical

# Mengubah label validasi menjadi format one-hot encoding
val_labels = to_categorical(val_labels, num_classes=5)

# Mengubah label pengujian menjadi format one-hot encoding
test_labels = to_categorical(test_labels, num_classes=5)

In [26]:
# Membangun model CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(5, activation='softmax'))  # Jumlah unit sesuai dengan jumlah kelas


In [27]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
val_images.shape

(10, 150, 150, 3, 5)

In [28]:
epochs = 10
history = model.fit(
    train_images,
    train_labels,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(val_images, val_labels)
)

Epoch 1/10


ValueError: in user code:

    File "/Users/aldalatifah/dev/my_test_project/myvenv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/Users/aldalatifah/dev/my_test_project/myvenv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/aldalatifah/dev/my_test_project/myvenv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/Users/aldalatifah/dev/my_test_project/myvenv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/aldalatifah/dev/my_test_project/myvenv/lib/python3.10/site-packages/keras/src/engine/training.py", line 1139, in compute_loss
        return self.compiled_l